In [1]:
%cd /content/
%rm -rf LLaMA-Factory
!git clone --depth 1 https://github.com/hiyouga/LLaMA-Factory.git
%cd LLaMA-Factory
%ls
!pip install torch==2.3.1 torchvision==0.18.1 torchaudio==2.3.1
!pip uninstall -y jax
!pip install -e .[torch,bitsandbytes,liger-kernel]

/content
Cloning into 'LLaMA-Factory'...
remote: Enumerating objects: 315, done.
remote: Counting objects: 100% (315/315), done.
remote: Compressing objects: 100% (239/239), done.
remote: Total 315 (delta 78), reused 195 (delta 63), pack-reused 0 (from 0)
Receiving objects: 100% (315/315), 9.00 MiB | 4.94 MiB/s, done.
Resolving deltas: 100% (78/78), done.
/content/LLaMA-Factory
assets/       docker/      LICENSE      pyproject.toml  requirements.txt  src/
CITATION.cff  evaluation/  Makefile     README.md       scripts/          tests/
data/         examples/    MANIFEST.in  README_zh.md    setup.py
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.1/779.1 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 68.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 96.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 110.1 MB/s eta 0:00:00


In [2]:
import json

In [6]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [7]:
!pip install datasets

In [8]:
!pip install bitsandbytes

In [9]:
from datasets import load_dataset
import json

# Load the dataset
dataset = load_dataset("argilla/ultrafeedback-binarized-preferences", split="train")

# Save the dataset locally
dataset.save_to_disk("ultrafeedback_dataset")

# Print the first example and column names to verify the structure
print(dataset[0])
print(dataset.column_names)

Generating train split:   0%|          | 0/63619 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/63619 [00:00<?, ? examples/s]

{'source': 'evol_instruct', 'instruction': 'Can you write a C++ program that prompts the user to enter the name of a country and checks if it borders the Mediterranean Sea? Here\'s some starter code to help you out:\n#include <iostream>\n#include <string>\nusing namespace std;\nint main() {\n    string country;\n    // prompt user for input\n    cout << "Enter the name of a country: ";\n    cin >> country;\n    // check if country borders the Mediterranean Sea\n    // [C++ code]\n    return 0;\n}', 'chosen_response': 'Here\'s a C++ program that prompts the user to enter the name of a country and checks if it borders the Mediterranean Sea:\n\n#include <iostream>\n#include <string>\n#include <set>\n#include <map>\n#include <algorithm>\n\nusing namespace std;\n\nint main() {\n    // store countries and their bordering seas in a map\n    map<string, set<string>> countries;\n    countries["Algeria"] = {"Mediterranean Sea", "North African Coast"};\n    countries["France"] = {"Mediterranean S

In [43]:
dataset_info = {
    "ultrafeedback": {
        "hf_hub_url": "argilla/ultrafeedback-binarized-preferences",
        "columns": {
            "prompt": "instruction",
            "chosen": "chosen_response",
            "rejected": "rejected_response"
        }
    }
}

with open("dataset_info.json", "w") as f:
    json.dump(dataset_info, f, indent=2)

In [11]:
sft_args = {
    "stage": "sft",
    "model_name_or_path": "meta-llama/Llama-2-7b-hf",
    "dataset": "ultrafeedback",
    "dataset_dir": ".",
    "template": "alpaca",
    "finetuning_type": "lora",
    "lora_target": "q_proj,v_proj",
    "output_dir": "llama2_sft_lora_ultrafeedback",
    "overwrite_output_dir": True,
    "num_train_epochs": 1,
    "max_samples": 10000,
    "per_device_train_batch_size": 4,
    "gradient_accumulation_steps": 4,
    "learning_rate": 2e-4,
    "lr_scheduler_type": "cosine",
    "warmup_ratio": 0.03,
    "logging_steps": 10,
    "save_steps": 1000,
    "eval_steps": 200,
    "evaluation_strategy": "steps",
    "max_grad_norm": 0.3,
    "val_size": 0.05,
    "preprocessing_num_workers": 4,
    "quantization_bit": 4
}

with open("sft_config.json", "w") as f:
    json.dump(sft_args, f, indent=2)

# Base Model - SFT

In [12]:
!llamafactory-cli train sft_config.json

2024-10-14 22:40:47.004610: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-14 22:40:47.344937: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-14 22:40:47.443142: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-14 22:40:47.994861: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-14 22:40:50.370044: W tensorflow/compiler/tf2

In [25]:
!CUDA_VISIBLE_DEVICES=0 python src/webui.py

2024-10-14 23:27:22.854300: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-14 23:27:22.875150: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-14 23:27:22.881419: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-14 23:27:22.896689: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-14 23:27:24.169357: W tensorflow/compiler/tf2

# DPO

Using Base model-SFT as adapter

In [44]:
import json

# DPO configuration
dpo_args = {
    "stage": "dpo",
    "model_name_or_path": "google/gemma-2b-it",  # Base model path
    "adapter_name_or_path": "/content/LLaMA-Factory/saves/Gemma-2B-Instruct/lora/train_2024-10-14-23-27-37",  # Path to the SFT LoRA adapter
    "dataset": "ultrafeedback",
    "dataset_dir": ".",
    "template": "gemma",
    "finetuning_type": "lora",
    "lora_target": "q_proj,v_proj",
    "output_dir": "/content/LLaMA-Factory/saves/Gemma-2B-Instruct/lora/dpo_output",
    "overwrite_output_dir": True,
    "num_train_epochs": 1,
    "max_samples": 10000,
    "per_device_train_batch_size": 4,
    "gradient_accumulation_steps": 4,
    "learning_rate": 1e-5,
    "lr_scheduler_type": "cosine",
    "warmup_ratio": 0.03,
    "logging_steps": 10,
    "save_steps": 1000,
    "eval_steps": 200,
    "evaluation_strategy": "steps",
    "max_grad_norm": 0.3,
    "val_size": 0.05,
    "preprocessing_num_workers": 4,
    "quantization_bit": 4
}

with open("dpo_config.json", "w") as f:
    json.dump(dpo_args, f, indent=2)



In [45]:
# Run DPO training
!llamafactory-cli train dpo_config.json

2024-10-15 02:21:58.618604: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-15 02:21:58.640403: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-15 02:21:58.648458: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-15 02:21:58.663626: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-15 02:22:00.212799: W tensorflow/compiler/tf2

In [38]:
!CUDA_VISIBLE_DEVICES=0 python src/webui.py

2024-10-15 01:54:56.160396: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-15 01:54:56.180596: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-15 01:54:56.187431: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-15 01:54:56.202628: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-15 01:54:57.472568: W tensorflow/compiler/tf2